In [ ]:
!pip install -q ensemble-boxes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# define root path
root_path = '/content/drive/My Drive/AI PROJECTS/Kaggle/VinBigData Chest X-Ray/'

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from ensemble_boxes import *

In [ ]:
meta_path = root_path + 'data/train.csv'

Merge boxes

In [ ]:
# ===============================
# Default WBF config (you can change these)
iou_thr = 0.5
skip_box_thr = 0.0001
sigma = 0.1
# ===============================

# Loading the train DF
df = pd.read_csv(meta_path)
df.fillna(0, inplace=True)
df.loc[df["class_id"] == 14, ['x_max', 'y_max']] = 1.0

results = []
image_ids = df["image_id"].unique()

for image_id in tqdm(image_ids, total=len(image_ids)):

    # All annotations for the current image.
    data = df[df["image_id"] == image_id]
    data = data.reset_index(drop=True)

    annotations = {}
    weights = []

    # WBF expects the coordinates in 0-1 range.
    max_value = data.iloc[:, 4:].values.max()
    data.loc[:, ["x_min", "y_min", "x_max", "y_max"]] = data.iloc[:, 4:] / max_value

    # Loop through all of the annotations
    for idx, row in data.iterrows():

        rad_id = row["rad_id"]

        if rad_id not in annotations:
            annotations[rad_id] = {
                "boxes_list": [],
                "scores_list": [],
                "labels_list": [],
            }

            # We consider all of the radiologists as equal.
            weights.append(1.0)

        annotations[rad_id]["boxes_list"].append([row["x_min"], row["y_min"], row["x_max"], row["y_max"]])
        annotations[rad_id]["scores_list"].append(1.0)
        annotations[rad_id]["labels_list"].append(row["class_id"])

    boxes_list = []
    scores_list = []
    labels_list = []

    for annotator in annotations.keys():
        boxes_list.append(annotations[annotator]["boxes_list"])
        scores_list.append(annotations[annotator]["scores_list"])
        labels_list.append(annotations[annotator]["labels_list"])

    # Calculate WBF
    boxes, scores, labels = weighted_boxes_fusion(
        boxes_list,
        scores_list,
        labels_list,
        weights=weights,
        iou_thr=iou_thr,
        skip_box_thr=skip_box_thr
    )

    for idx, box in enumerate(boxes):
        results.append({
            "image_id": image_id,
            "class_id": int(labels[idx]),
            "rad_id": "wbf",
            "x_min": box[0] * max_value,
            "y_min": box[1] * max_value,
            "x_max": box[2] * max_value,
            "y_max": box[3] * max_value,
        })

results = pd.DataFrame(results)

100%|██████████| 15000/15000 [02:14<00:00, 111.78it/s]


export new csv file

In [ ]:
classes = {
    0: 'Aortic enlargement',
    1: 'Atelectasis',
    2: 'Calcification',
    3: 'Cardiomegaly',
    4: 'Consolidation',
    5: 'ILD',
    6: 'Infiltration',
    7: 'Lung Opacity',
    8: 'Nodule/Mass',
    9: 'Other lesion',
    10: 'Pleural effusion',
    11: 'Pleural thickening',
    12: 'Pneumothorax',
    13: 'Pulmonary fibrosis',
    14: 'No finding'
}

In [ ]:
colors = {
    "Aortic enlargement": (255, 0, 0),
    "Atelectasis": (0, 255, 0),
    "Calcification": (0, 0, 255),
    "Cardiomegaly": (255, 255, 0),
    "Consolidation": (255, 0, 255),
    "ILD": (0, 255, 255),
    "Infiltration": (255, 255, 255),
    "Lung Opacity": (255, 100, 50),
    "Nodule/Mass": (255, 50, 100),
    "Other lesion": (100, 10, 255),
    "Pleural effusion": (10, 100, 255),
    "Pleural thickening": (10, 50, 50),
    "Pneumothorax": (90, 90, 32),
    "Pulmonary fibrosis": (145, 32, 23),
}

In [ ]:
df.shape

(67914, 8)

In [ ]:
results.shape

(34510, 7)

In [ ]:
results.head()

,image_id,class_id,rad_id,x_min,y_min,x_max,y_max
0,50a418190bc3fb1ef1633bf9678929b3,14,wbf,0.000000,0.000000,1.000000,1.000000
1,21a10246a5ec7af151081d0cd6d65dc9,14,wbf,0.000000,0.000000,1.000000,1.000000
2,9a5094b2563a1ef3ff50dc5c7ff71345,3,wbf,690.666676,1354.333235,1658.666714,1797.666677
3,9a5094b2563a1ef3ff50dc5c7ff71345,0,wbf,1052.000000,715.000000,1299.000000,966.000000
4,9a5094b2563a1ef3ff50dc5c7ff71345,11,wbf,1789.000000,1729.000000,1875.000000,1992.000000


In [ ]:
df.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,0.0,0.0,1.0,1.0
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,0.0,0.0,1.0,1.0
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,0.0,0.0,1.0,1.0


In [ ]:
# add class_name column into results
results['class_name'] = results['class_id'].apply(lambda x: classes[x])

In [ ]:
results.head()

,image_id,class_id,rad_id,x_min,y_min,x_max,y_max,class_name
0,50a418190bc3fb1ef1633bf9678929b3,14,wbf,0.000000,0.000000,1.000000,1.000000,No finding
1,21a10246a5ec7af151081d0cd6d65dc9,14,wbf,0.000000,0.000000,1.000000,1.000000,No finding
2,9a5094b2563a1ef3ff50dc5c7ff71345,3,wbf,690.666676,1354.333235,1658.666714,1797.666677,Cardiomegaly
3,9a5094b2563a1ef3ff50dc5c7ff71345,0,wbf,1052.000000,715.000000,1299.000000,966.000000,Aortic enlargement
4,9a5094b2563a1ef3ff50dc5c7ff71345,11,wbf,1789.000000,1729.000000,1875.000000,1992.000000,Pleural thickening


In [ ]:
results.to_csv(root_path + 'data/merged_train.csv', index=False)

draw boxes to check Weighted Box Fusion

In [ ]:
labeled_images_folder = root_path + 'data/small_dataset/train_images_abnormalities'
source_folder = root_path + 'data/train_images'
merged_labeled_images_folder = root_path + 'data/small_dataset/merged_train_images_abnormalities'
merged_meta_path = root_path + 'data/merged_train.csv'

In [ ]:
count_file = 0
for f in os.listdir(labeled_images_folder):
  src_path = os.path.join(source_folder, f)
  saved_path = os.path.join(merged_labeled_images_folder, f)
  # print(saved_path)

  boxes = results[results['image_id'] == f[:-4]][['class_name', 'x_min', 'y_min', 'x_max', 'y_max']].values.tolist()

  img = cv2.imread(src_path)
  for box in boxes:
    cls = box[0]
    if cls != "No finding":
      x1, y1, x2, y2 = int(box[1]), int(box[2]), int(box[3]), int(box[4]) 
      cv2.rectangle(img, (x1, y1), (x2, y2), colors[cls], 2)

      textOrg = (x1, y1)
      cv2.putText(img, cls, textOrg, cv2.FONT_HERSHEY_DUPLEX, 2, colors[cls], 2)
    
  count_file += 1
  print(count_file)
  ##### save image #####
  cv2.imwrite(saved_path, img)
    ######################

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
